In [3]:
# -------------------------------------------
# MPIN Strength Checker - Installation Script
# -------------------------------------------
# This cell installs all required dependencies for the
# Gradio-based MPIN checker in Google Colab.
# Just run this cell before running the main app!
# -------------------------------------------

# Install Gradio (for the modern web UI)
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
# -------------------------------------------
# MPIN Strength Checker - Features & Explanation
# -------------------------------------------
# Features:
# 1. Modern, user-friendly web UI built with Gradio, including clear labels, placeholders, rounded corners, and color themes.
# 2. Checks the strength of a 4 or 6-digit MPIN (Mobile PIN) against:
#    - Commonly used PINs (like "1234", "0000", etc.).
#    - Sequential number patterns (like "1234", "654321").
#    - Repeated digits and repeated groups (like "1111", "1212", "121212").
#    - Palindromic numbers (like "1221", "123321").
#    - Keypad patterns (straight lines, "L" shapes, etc. as used on a numeric keypad).
#    - PINs based on your Date of Birth, Anniversary, or Spouse’s DOB (in various formats).
#    - PINs that appear in your phone number.
# 3. Gives clear reasons why a PIN is considered weak (with user-friendly explanations).
# 4. Suggests 2-3 strong, randomly generated alternative PINs that avoid all the above weaknesses.
# 5. All user input fields are clearly labeled (with format guidance) and the UI is visually modern and accessible.
# 6. The code is fully commented for clarity and easy understanding.
# 7. Uses only Python and Gradio—no accounts, no authentication, no browser extensions required.
# 8. Designed to work seamlessly in Google Colab or on your local machine.
#
# Explanation:
# This project helps users choose a secure MPIN for financial and mobile apps by checking if the PIN is easily guessable or vulnerable.
# It uses multiple detection techniques to catch patterns and personally related numbers that make a PIN insecure.
# The app guides the user with clear feedback and suggestions, making it easy for everyone to set a strong MPIN.
#
# Author: [Your Name]
# Date: 2025-05-01
# Reference: Inspired by real-world banking security guidelines and community best-practices.
# -------------------------------------------

## **Test Cases**

## **with gui**

In [4]:
# MPIN Strength Checker - Data-Driven, Colab + GitHub Resource Version
# --------------------------------------------------------------------
# This version downloads resource files from a GitHub repo on startup,
# so files are always available in Colab, even after a session reset.
# Update GITHUB_RAW_URL_PREFIX to your own repo as needed.

import gradio as gr
from itertools import permutations
import time
import os

# ------------------ Download Resource Files from GitHub ------------------

# CHANGE THIS to your repo's raw content URL (without trailing slash)
GITHUB_RAW_URL_PREFIX = "https://github.com/YooshaMirza/onebanc-project/tree/main/resources"

# Download resource files from GitHub
def download_resource_files():
    files = [
        "common_4digit_mpins.txt",
        "common_6digit_mpins.txt",
        "keypad_patterns.txt"
    ]
    for fname in files:
        url = f"{GITHUB_RAW_URL_PREFIX}/{fname}"
        os.system(f"wget -q -O {fname} {url}")

download_resource_files()

# ------------------ Data Loading Utilities ------------------

def load_set_from_file(filepath):
    with open(filepath, "r") as f:
        return set(line.strip() for line in f if line.strip())

COMMON_4DIGIT_MPINS = load_set_from_file("common_4digit_mpins.txt")
COMMON_6DIGIT_MPINS = load_set_from_file("common_6digit_mpins.txt")
KEYPAD_PATTERNS = load_set_from_file("keypad_patterns.txt")

KEYPAD_PATTERNS_6 = set()
for pattern in KEYPAD_PATTERNS:
    if len(pattern) == 3:
        KEYPAD_PATTERNS_6.add(pattern * 2)
    elif len(pattern) == 4:
        KEYPAD_PATTERNS_6.add(pattern + pattern[:2])
        KEYPAD_PATTERNS_6.add(pattern[::-1] + pattern[::-1][:2])
    elif len(pattern) == 6:
        KEYPAD_PATTERNS_6.add(pattern)

# ------------------ Core Logic (No Hardcoded Lists) ------------------

def extract_date_parts(date):
    parts = date.strip().split('-')
    if len(parts) != 3 or not all(p.isdigit() for p in parts):
        return '', '', ''
    if len(parts[0]) == 4:
        y, m, d = parts
    elif len(parts[2]) == 4:
        d, m, y = parts
    else:
        return '', '', ''
    return y.zfill(4), m.zfill(2), d.zfill(2)

def generate_all_4digit_weak_patterns(date):
    y, m, d = extract_date_parts(date)
    if not (y and m and d):
        return set()
    year4, year2 = y, y[-2:]
    month2, month1 = m, str(int(m))
    day2, day1 = d, str(int(d))
    fields = [year4, year2, month2, month1, day2, day1]
    patterns = set()
    for combo_len in (2, 3):
        for combo in permutations(fields, combo_len):
            joined = ''.join(combo)
            if len(joined) == 4:
                patterns.add(joined)
    concat_strs = [
        year4 + month2 + day2, day2 + month2 + year4, month2 + day2 + year4,
        year4 + day2 + month2, month2 + year4 + day2, day2 + year4 + month2,
        year2 + month2 + day2, day2 + month2 + year2, month2 + day2 + year2,
        year2 + day2 + month2, month2 + year2 + day2, day2 + year2 + month2,
        day2 + month1 + year2, month1 + day2 + year2
    ]
    for s in concat_strs:
        for i in range(len(s) - 3):
            patterns.add(s[i:i+4])
    for x in range(10):
        patterns.add(f"{x}{day2}{month1}")
    if len(day2+month1) == 4:
        patterns.add(day2+month1)
    if len(month1+day2) == 4:
        patterns.add(month1+day2)
    combos = [
        month1+day2+year2, day2+month1+year2, year2+month1+day2, day2+year2+month1,
        month2+day2+year2, day2+month2+year2, year2+month2+day2, day2+year2+month2
    ]
    for c in combos:
        if len(c) == 4:
            patterns.add(c)
    patterns |= {p[::-1] for p in patterns}
    return {p for p in patterns if p.isdigit() and len(p) == 4}

def generate_all_6digit_weak_patterns(date):
    y, m, d = extract_date_parts(date)
    if not (y and m and d):
        return set()
    year4, year2 = y, y[-2:]
    month2, month1 = m, str(int(m))
    day2, day1 = d, str(int(d))
    fields = [year4, year2, month2, month1, day2, day1]
    patterns = set()
    for combo_len in (2, 3):
        for combo in permutations(fields, combo_len):
            joined = ''.join(combo)
            if len(joined) == 6:
                patterns.add(joined)
    concat_strs = [
        year4 + month2 + day2, day2 + month2 + year4, month2 + day2 + year4,
        year4 + day2 + month2, month2 + year4 + day2, day2 + year4 + month2,
        year2 + month2 + day2, day2 + month2 + year2, month2 + day2 + year2,
        year2 + day2 + month2, month2 + year2 + day2, day2 + year2 + month2,
        day2 + month1 + year2, month1 + day2 + year2
    ]
    for s in concat_strs:
        for i in range(len(s) - 5):
            patterns.add(s[i:i+6])
    patterns |= {p[::-1] for p in patterns}
    return {p for p in patterns if p.isdigit() and len(p) == 6}

def is_commonly_used_mpin(mpin, c4, c6):
    if len(mpin) == 4:
        return mpin in c4
    elif len(mpin) == 6:
        return mpin in c6
    return False

def is_sequential(mpin):
    inc = ''.join(str((int(mpin[0]) + i) % 10) for i in range(len(mpin)))
    dec = ''.join(str((int(mpin[0]) - i) % 10) for i in range(len(mpin)))
    return mpin == inc or mpin == dec

def is_repeated(mpin):
    if len(set(mpin)) == 1:
        return True
    if len(mpin) % 2 == 0:
        half = len(mpin) // 2
        if mpin[:half] == mpin[half:]:
            return True
        if mpin[:2] * (len(mpin)//2) == mpin:
            return True
    return False

def is_palindrome(mpin):
    return mpin == mpin[::-1]

def is_simple_keypad_pattern(mpin, keypad, keypad6):
    if len(mpin) == 4:
        return mpin in keypad or mpin[::-1] in keypad
    elif len(mpin) == 6:
        return mpin in keypad6 or mpin[::-1] in keypad6
    return False

def personal_data_leak(mpin, phone_number):
    if phone_number and mpin in phone_number.replace(" ", "").replace("-", ""):
        return True
    return False

def recommend_alternate_mpins(mpin, demographic_patterns, phone_number, avoid_patterns, keypad, keypad6):
    from random import randint, seed
    seed(int(mpin)+int(time.time()))
    recommendations = []
    attempts = 0
    while len(recommendations) < 3 and attempts < 1000:
        candidate = ""
        while len(candidate) < len(mpin):
            digit = str(randint(0, 9))
            if candidate and digit == candidate[-1]:
                continue
            candidate += digit
        if candidate in avoid_patterns:
            attempts += 1
            continue
        if candidate in demographic_patterns:
            attempts += 1
            continue
        if phone_number and candidate in phone_number:
            attempts += 1
            continue
        if is_sequential(candidate) or is_repeated(candidate) or is_palindrome(candidate):
            attempts += 1
            continue
        if is_simple_keypad_pattern(candidate, keypad, keypad6):
            attempts += 1
            continue
        if candidate in recommendations:
            attempts += 1
            continue
        recommendations.append(candidate)
    return recommendations

def get_additional_weakness(mpin, keypad, keypad6):
    reasons = []
    if is_sequential(mpin):
        reasons.append("SEQUENTIAL_PATTERN")
    if is_repeated(mpin):
        reasons.append("REPEATED_PATTERN")
    if is_palindrome(mpin):
        reasons.append("PALINDROME")
    if (len(mpin) == 4 and mpin.isdigit() and (1800 <= int(mpin) <= 2099)):
        reasons.append("YEAR_ONLY")
    if is_simple_keypad_pattern(mpin, keypad, keypad6):
        reasons.append("KEYPAD_PATTERN")
    return reasons

def mpin_strength(mpin, demographics, phone, c4, c6, keypad, keypad6):
    reasons = []
    pin_length = len(mpin)
    patterns = set()
    for label, date in demographics.items():
        if date:
            if pin_length == 4:
                patterns |= generate_all_4digit_weak_patterns(date)
            elif pin_length == 6:
                patterns |= generate_all_6digit_weak_patterns(date)
    if is_commonly_used_mpin(mpin, c4, c6):
        reasons.append("COMMONLY_USED")
    if "dob" in demographics and demographics["dob"]:
        if mpin in generate_all_4digit_weak_patterns(demographics["dob"]) or mpin in generate_all_6digit_weak_patterns(demographics["dob"]):
            reasons.append("DEMOGRAPHIC_DOB_SELF")
    if "spouse_dob" in demographics and demographics["spouse_dob"]:
        if mpin in generate_all_4digit_weak_patterns(demographics["spouse_dob"]) or mpin in generate_all_6digit_weak_patterns(demographics["spouse_dob"]):
            reasons.append("DEMOGRAPHIC_DOB_SPOUSE")
    if "anniversary" in demographics and demographics["anniversary"]:
        if mpin in generate_all_4digit_weak_patterns(demographics["anniversary"]) or mpin in generate_all_6digit_weak_patterns(demographics["anniversary"]):
            reasons.append("DEMOGRAPHIC_ANNIVERSARY")
    reasons += get_additional_weakness(mpin, keypad, keypad6)
    if personal_data_leak(mpin, phone):
        reasons.append("PERSONAL_DATA_LEAK (phone)")
    return ("WEAK", reasons, patterns) if reasons else ("STRONG", [], patterns)

# ----- Gradio UI & Handler (No Hardcoding) -----
def mpin_checker_gradio(mpin, dob, spouse_dob, anniversary, phone):
    if not mpin or len(mpin) not in (4, 6) or not mpin.isdigit():
        return "❌ Please enter a valid 4 or 6 digit numeric MPIN.", "", ""
    demographics = {"dob": dob, "spouse_dob": spouse_dob, "anniversary": anniversary}
    strength, reasons, weak_patterns = mpin_strength(
        mpin, demographics, phone,
        COMMON_4DIGIT_MPINS, COMMON_6DIGIT_MPINS,
        KEYPAD_PATTERNS, KEYPAD_PATTERNS_6
    )
    if not reasons:
        return f"✅ MPIN: {mpin}\n\nThis is a strong MPIN! 🚀", "", ""
    else:
        REASON_MAP = {
            "COMMONLY_USED": "It's a commonly used PIN.",
            "DEMOGRAPHIC_DOB_SELF": "Based on your Date of Birth.",
            "DEMOGRAPHIC_DOB_SPOUSE": "Based on your Spouse's DOB.",
            "DEMOGRAPHIC_ANNIVERSARY": "Based on your Anniversary.",
            "SEQUENTIAL_PATTERN": "It's a sequential pattern (e.g. 1234).",
            "REPEATED_PATTERN": "It repeats a digit or group (e.g. 1212, 1111).",
            "PALINDROME": "It's a palindrome.",
            "YEAR_ONLY": "It's a year.",
            "KEYPAD_PATTERN": "It's a keypad pattern (straight line, etc).",
            "PERSONAL_DATA_LEAK (phone)": "It appears in your phone number."
        }
        reasons_str = "Weak because:\n" + "\n".join(f"- {REASON_MAP.get(r, r)}" for r in reasons)
        recs = recommend_alternate_mpins(
            mpin, weak_patterns, phone, avoid_patterns=weak_patterns,
            keypad=KEYPAD_PATTERNS, keypad6=KEYPAD_PATTERNS_6
        )
        recs_str = "Recommended strong PINs:\n" + "\n".join(f"• {r}" for r in recs)
        return f"⚠️ MPIN: {mpin}\n", reasons_str, recs_str

# ---------- Gradio UI definition ----------

with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", neutral_hue="slate")) as iface:
    gr.Markdown("""
    <div style="text-align:center">
        <h1 style="color:#2563eb; font-family:Segoe UI,Arial,sans-serif; margin-bottom:0.1em; letter-spacing:1px;">
            🔒 MPIN Strength Checker
        </h1>
        <p style="color:#64748b; font-size:1.12em; margin-top:0.2em">
            <b>Check your MPIN against modern security standards.</b>
        </p>
    </div>
    """)
    with gr.Row():
        with gr.Column(scale=2):
            mpin_box = gr.Textbox(
                label="MPIN (4 or 6 digits)",
                placeholder="e.g. 1379 or 123456",
                type="password",
                max_lines=1,
                container=True
            )
            dob_box = gr.Textbox(
                label="Your Date of Birth",
                placeholder="Format: YYYY-MM-DD or DD-MM-YYYY",
                max_lines=1,
                container=True
            )
            spouse_box = gr.Textbox(
                label="Spouse's Date of Birth (optional)",
                placeholder="Format: YYYY-MM-DD or DD-MM-YYYY",
                max_lines=1,
                container=True
            )
            anniv_box = gr.Textbox(
                label="Anniversary (optional)",
                placeholder="Format: YYYY-MM-DD or DD-MM-YYYY",
                max_lines=1,
                container=True
            )
            phone_box = gr.Textbox(
                label="Phone Number (optional)",
                placeholder="Enter digits only",
                max_lines=1,
                container=True
            )
            btn = gr.Button("Check MPIN Strength", elem_id="submit-btn", scale=2)
        with gr.Column(scale=3):
            mpin_strength_out = gr.Textbox(
                label="MPIN Strength",
                max_lines=2,
                show_copy_button=False,
                interactive=False,
                elem_id="strength-box",
                container=True
            )
            why_weak_out = gr.Textbox(
                label="Why weak?",
                max_lines=8,
                show_copy_button=False,
                interactive=False,
                elem_id="why-box",
                container=True
            )
            recs_out = gr.Textbox(
                label="Recommended Alternatives",
                max_lines=6,
                show_copy_button=False,
                interactive=False,
                elem_id="rec-box",
                container=True
            )
    btn.click(
        mpin_checker_gradio,
        [mpin_box, dob_box, spouse_box, anniv_box, phone_box],
        [mpin_strength_out, why_weak_out, recs_out]
    )
    iface.css = """
    textarea, input, #submit-btn, .gr-button {
        border-radius: 16px !important;
        font-size: 1.08em !important;
        padding: 0.7em 1em !important;
    }
    .gr-textbox, .gr-box, #strength-box, #why-box, #rec-box {
        border-radius: 18px !important;
        border: 2px solid #2563eb22 !important;
        background: #f6f9ff !important;
        font-family: Segoe UI, Arial, sans-serif !important;
    }
    label, .gr-label {
        font-weight: 500 !important;
        color: #2563eb !important;
        letter-spacing: 0.5px !important;
    }
    #submit-btn {
        background: linear-gradient(90deg,#2563eb 40%,#1e40af 100%) !important;
        color: #fff !important;
        border: none !important;
        font-size: 1.13em !important;
        box-shadow: 0 2px 8px #2563eb22 !important;
        margin-top: 2.2em;
    }
    """

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5a14dd0b62be8658db.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## **without gui**

In [6]:
# MPIN Strength Checker - Data-Driven, Command-Line Version (No GUI)
# ------------------------------------------------------------------
# Downloads resource files from a GitHub repo on startup (if not present),
# then allows command-line checking and batch testing of MPINs.
# Update GITHUB_RAW_URL_PREFIX to your own repo as needed.

import os
from itertools import permutations
import time

# CHANGE THIS to your repo's raw content URL (without trailing slash)
GITHUB_RAW_URL_PREFIX = "https://github.com/YooshaMirza/onebanc-project/tree/main/resources"

# Download resource files from GitHub if not present
def download_resource_files():
    files = [
        "common_4digit_mpins.txt",
        "common_6digit_mpins.txt",
        "keypad_patterns.txt"
    ]
    for fname in files:
        if not os.path.exists(fname):
            url = f"{GITHUB_RAW_URL_PREFIX}/{fname}"
            print(f"Downloading {fname} from {url} ...")
            os.system(f"wget -q -O {fname} {url}")

download_resource_files()

def load_set_from_file(filepath):
    with open(filepath, "r") as f:
        return set(line.strip() for line in f if line.strip())

COMMON_4DIGIT_MPINS = load_set_from_file("common_4digit_mpins.txt")
COMMON_6DIGIT_MPINS = load_set_from_file("common_6digit_mpins.txt")
KEYPAD_PATTERNS = load_set_from_file("keypad_patterns.txt")

KEYPAD_PATTERNS_6 = set()
for pattern in KEYPAD_PATTERNS:
    if len(pattern) == 3:
        KEYPAD_PATTERNS_6.add(pattern * 2)
    elif len(pattern) == 4:
        KEYPAD_PATTERNS_6.add(pattern + pattern[:2])
        KEYPAD_PATTERNS_6.add(pattern[::-1] + pattern[::-1][:2])
    elif len(pattern) == 6:
        KEYPAD_PATTERNS_6.add(pattern)

def extract_date_parts(date):
    parts = date.strip().split('-')
    if len(parts) != 3 or not all(p.isdigit() for p in parts):
        return '', '', ''
    if len(parts[0]) == 4:
        y, m, d = parts
    elif len(parts[2]) == 4:
        d, m, y = parts
    else:
        return '', '', ''
    return y.zfill(4), m.zfill(2), d.zfill(2)

def generate_all_4digit_weak_patterns(date):
    y, m, d = extract_date_parts(date)
    if not (y and m and d):
        return set()
    year4, year2 = y, y[-2:]
    month2, month1 = m, str(int(m))
    day2, day1 = d, str(int(d))
    fields = [year4, year2, month2, month1, day2, day1]
    patterns = set()
    for combo_len in (2, 3):
        for combo in permutations(fields, combo_len):
            joined = ''.join(combo)
            if len(joined) == 4:
                patterns.add(joined)
    concat_strs = [
        year4 + month2 + day2, day2 + month2 + year4, month2 + day2 + year4,
        year4 + day2 + month2, month2 + year4 + day2, day2 + year4 + month2,
        year2 + month2 + day2, day2 + month2 + year2, month2 + day2 + year2,
        year2 + day2 + month2, month2 + year2 + day2, day2 + year2 + month2,
        day2 + month1 + year2, month1 + day2 + year2
    ]
    for s in concat_strs:
        for i in range(len(s) - 3):
            patterns.add(s[i:i+4])
    for x in range(10):
        patterns.add(f"{x}{day2}{month1}")
    if len(day2+month1) == 4:
        patterns.add(day2+month1)
    if len(month1+day2) == 4:
        patterns.add(month1+day2)
    combos = [
        month1+day2+year2, day2+month1+year2, year2+month1+day2, day2+year2+month1,
        month2+day2+year2, day2+month2+year2, year2+month2+day2, day2+year2+month2
    ]
    for c in combos:
        if len(c) == 4:
            patterns.add(c)
    patterns |= {p[::-1] for p in patterns}
    return {p for p in patterns if p.isdigit() and len(p) == 4}

def generate_all_6digit_weak_patterns(date):
    y, m, d = extract_date_parts(date)
    if not (y and m and d):
        return set()
    year4, year2 = y, y[-2:]
    month2, month1 = m, str(int(m))
    day2, day1 = d, str(int(d))
    fields = [year4, year2, month2, month1, day2, day1]
    patterns = set()
    for combo_len in (2, 3):
        for combo in permutations(fields, combo_len):
            joined = ''.join(combo)
            if len(joined) == 6:
                patterns.add(joined)
    concat_strs = [
        year4 + month2 + day2, day2 + month2 + year4, month2 + day2 + year4,
        year4 + day2 + month2, month2 + year4 + day2, day2 + year4 + month2,
        year2 + month2 + day2, day2 + month2 + year2, month2 + day2 + year2,
        year2 + day2 + month2, month2 + year2 + day2, day2 + year2 + month2,
        day2 + month1 + year2, month1 + day2 + year2
    ]
    for s in concat_strs:
        for i in range(len(s) - 5):
            patterns.add(s[i:i+6])
    patterns |= {p[::-1] for p in patterns}
    return {p for p in patterns if p.isdigit() and len(p) == 6}

def is_commonly_used_mpin(mpin, c4, c6):
    if len(mpin) == 4:
        return mpin in c4
    elif len(mpin) == 6:
        return mpin in c6
    return False

def is_sequential(mpin):
    inc = ''.join(str((int(mpin[0]) + i) % 10) for i in range(len(mpin)))
    dec = ''.join(str((int(mpin[0]) - i) % 10) for i in range(len(mpin)))
    return mpin == inc or mpin == dec

def is_repeated(mpin):
    if len(set(mpin)) == 1:
        return True
    if len(mpin) % 2 == 0:
        half = len(mpin) // 2
        if mpin[:half] == mpin[half:]:
            return True
        if mpin[:2] * (len(mpin)//2) == mpin:
            return True
    return False

def is_palindrome(mpin):
    return mpin == mpin[::-1]

def is_simple_keypad_pattern(mpin, keypad, keypad6):
    if len(mpin) == 4:
        return mpin in keypad or mpin[::-1] in keypad
    elif len(mpin) == 6:
        return mpin in keypad6 or mpin[::-1] in keypad6
    return False

def personal_data_leak(mpin, phone_number):
    if phone_number and mpin in phone_number.replace(" ", "").replace("-", ""):
        return True
    return False

def recommend_alternate_mpins(mpin, demographic_patterns, phone_number, avoid_patterns, keypad, keypad6):
    from random import randint, seed
    seed(int(mpin)+int(time.time()))
    recommendations = []
    attempts = 0
    while len(recommendations) < 3 and attempts < 1000:
        candidate = ""
        while len(candidate) < len(mpin):
            digit = str(randint(0, 9))
            if candidate and digit == candidate[-1]:
                continue
            candidate += digit
        if candidate in avoid_patterns:
            attempts += 1
            continue
        if candidate in demographic_patterns:
            attempts += 1
            continue
        if phone_number and candidate in phone_number:
            attempts += 1
            continue
        if is_sequential(candidate) or is_repeated(candidate) or is_palindrome(candidate):
            attempts += 1
            continue
        if is_simple_keypad_pattern(candidate, keypad, keypad6):
            attempts += 1
            continue
        if candidate in recommendations:
            attempts += 1
            continue
        recommendations.append(candidate)
    return recommendations

def get_additional_weakness(mpin, keypad, keypad6):
    reasons = []
    if is_sequential(mpin):
        reasons.append("SEQUENTIAL_PATTERN")
    if is_repeated(mpin):
        reasons.append("REPEATED_PATTERN")
    if is_palindrome(mpin):
        reasons.append("PALINDROME")
    if (len(mpin) == 4 and mpin.isdigit() and (1800 <= int(mpin) <= 2099)):
        reasons.append("YEAR_ONLY")
    if is_simple_keypad_pattern(mpin, keypad, keypad6):
        reasons.append("KEYPAD_PATTERN")
    return reasons

def mpin_strength(mpin, demographics, phone, c4, c6, keypad, keypad6):
    reasons = []
    pin_length = len(mpin)
    patterns = set()
    for label, date in demographics.items():
        if date:
            if pin_length == 4:
                patterns |= generate_all_4digit_weak_patterns(date)
            elif pin_length == 6:
                patterns |= generate_all_6digit_weak_patterns(date)
    if is_commonly_used_mpin(mpin, c4, c6):
        reasons.append("COMMONLY_USED")
    if "dob" in demographics and demographics["dob"]:
        if mpin in generate_all_4digit_weak_patterns(demographics["dob"]) or mpin in generate_all_6digit_weak_patterns(demographics["dob"]):
            reasons.append("DEMOGRAPHIC_DOB_SELF")
    if "spouse_dob" in demographics and demographics["spouse_dob"]:
        if mpin in generate_all_4digit_weak_patterns(demographics["spouse_dob"]) or mpin in generate_all_6digit_weak_patterns(demographics["spouse_dob"]):
            reasons.append("DEMOGRAPHIC_DOB_SPOUSE")
    if "anniversary" in demographics and demographics["anniversary"]:
        if mpin in generate_all_4digit_weak_patterns(demographics["anniversary"]) or mpin in generate_all_6digit_weak_patterns(demographics["anniversary"]):
            reasons.append("DEMOGRAPHIC_ANNIVERSARY")
    reasons += get_additional_weakness(mpin, keypad, keypad6)
    if personal_data_leak(mpin, phone):
        reasons.append("PERSONAL_DATA_LEAK (phone)")
    return ("WEAK", reasons, patterns) if reasons else ("STRONG", [], patterns)

def pretty_reasons(reasons):
    REASON_MAP = {
        "COMMONLY_USED": "It's a commonly used PIN.",
        "DEMOGRAPHIC_DOB_SELF": "Based on your Date of Birth.",
        "DEMOGRAPHIC_DOB_SPOUSE": "Based on your Spouse's DOB.",
        "DEMOGRAPHIC_ANNIVERSARY": "Based on your Anniversary.",
        "SEQUENTIAL_PATTERN": "It's a sequential pattern (e.g. 1234).",
        "REPEATED_PATTERN": "It repeats a digit or group (e.g. 1212, 1111).",
        "PALINDROME": "It's a palindrome.",
        "YEAR_ONLY": "It's a year.",
        "KEYPAD_PATTERN": "It's a keypad pattern (straight line, etc).",
        "PERSONAL_DATA_LEAK (phone)": "It appears in your phone number."
    }
    return "\n".join(f"- {REASON_MAP.get(r, r)}" for r in reasons)

def run_interactive():
    print("MPIN Strength Checker (Command-Line, Data-Driven)")
    print("-------------------------------------------------")
    mpin = input("Enter MPIN (4 or 6 digits): ").strip()
    dob = input("Your Date of Birth (YYYY-MM-DD or DD-MM-YYYY): ").strip()
    spouse_dob = input("Spouse's DOB (optional): ").strip()
    anniversary = input("Anniversary (optional): ").strip()
    phone = input("Phone Number (optional): ").strip()
    demographics = {"dob": dob, "spouse_dob": spouse_dob, "anniversary": anniversary}
    if not mpin or len(mpin) not in (4, 6) or not mpin.isdigit():
        print("❌ Please enter a valid 4 or 6 digit numeric MPIN.")
        return
    strength, reasons, weak_patterns = mpin_strength(
        mpin, demographics, phone,
        COMMON_4DIGIT_MPINS, COMMON_6DIGIT_MPINS,
        KEYPAD_PATTERNS, KEYPAD_PATTERNS_6
    )
    if not reasons:
        print(f"✅ MPIN: {mpin}")
        print("This is a strong MPIN! 🚀")
    else:
        print(f"⚠️ MPIN: {mpin}")
        print("Weak because:")
        print(pretty_reasons(reasons))
        recs = recommend_alternate_mpins(
            mpin, weak_patterns, phone, avoid_patterns=weak_patterns,
            keypad=KEYPAD_PATTERNS, keypad6=KEYPAD_PATTERNS_6
        )
        print("Recommended strong PINs:")
        for r in recs:
            print(f"• {r}")

if __name__ == "__main__":
    run_interactive()

MPIN Strength Checker (Command-Line, Data-Driven)
-------------------------------------------------
Enter MPIN (4 or 6 digits): 1234
Your Date of Birth (YYYY-MM-DD or DD-MM-YYYY): 
Spouse's DOB (optional): 
Anniversary (optional): 
Phone Number (optional): 
⚠️ MPIN: 1234
Weak because:
- It's a sequential pattern (e.g. 1234).
Recommended strong PINs:
• 7458
• 5726
• 0174


In [8]:
# MPIN Strength Checker - 50+ Complex Test Cases (Tabular Output, Data-Driven)
# ----------------------------------------------------------------------------
# This script expects mpin_strength(), helper logic, and resource sets
# (COMMON_4DIGIT_MPINS, COMMON_6DIGIT_MPINS, KEYPAD_PATTERNS, KEYPAD_PATTERNS_6)
# to be already loaded in your environment.

import pandas as pd

def run_test_case_to_row(mpin, dob="", spouse_dob="", anniversary="", phone=""):
    demographics = {
        "dob": dob,
        "spouse_dob": spouse_dob,
        "anniversary": anniversary
    }
    if not mpin or len(mpin) not in (4, 6) or not mpin.isdigit():
        return {
            "MPIN": mpin,
            "DOB": dob,
            "Spouse DOB": spouse_dob,
            "Anniv": anniversary,
            "Phone": phone,
            "Strength": "INVALID",
            "Reasons": "Invalid MPIN: must be 4 or 6 digits and numeric."
        }
    # Pass the required sets!
    strength, reasons, _ = mpin_strength(
        mpin, demographics, phone,
        COMMON_4DIGIT_MPINS, COMMON_6DIGIT_MPINS,
        KEYPAD_PATTERNS, KEYPAD_PATTERNS_6
    )
    return {
        "MPIN": mpin,
        "DOB": dob,
        "Spouse DOB": spouse_dob,
        "Anniv": anniversary,
        "Phone": phone,
        "Strength": strength,
        "Reasons": ", ".join(reasons) if reasons else "(strong)"
    }

# List to store result rows
results = []

# 1. Classic sequential patterns
results.append(run_test_case_to_row("1234"))
results.append(run_test_case_to_row("2345"))
results.append(run_test_case_to_row("6789"))
results.append(run_test_case_to_row("7890"))
results.append(run_test_case_to_row("4321"))
results.append(run_test_case_to_row("2109"))
results.append(run_test_case_to_row("0987"))
results.append(run_test_case_to_row("3210"))
results.append(run_test_case_to_row("123456"))
results.append(run_test_case_to_row("654321"))

# 2. Repeated digits and mirrored patterns
results.append(run_test_case_to_row("1111"))
results.append(run_test_case_to_row("2222"))
results.append(run_test_case_to_row("3333"))
results.append(run_test_case_to_row("4444"))
results.append(run_test_case_to_row("1212"))
results.append(run_test_case_to_row("3434"))
results.append(run_test_case_to_row("5656"))
results.append(run_test_case_to_row("121212"))
results.append(run_test_case_to_row("909090"))
results.append(run_test_case_to_row("9090"))

# 3. Palindromes and mirrored years
results.append(run_test_case_to_row("1221"))
results.append(run_test_case_to_row("2112"))
results.append(run_test_case_to_row("3443"))
results.append(run_test_case_to_row("123321"))
results.append(run_test_case_to_row("456654"))
results.append(run_test_case_to_row("4004"))
results.append(run_test_case_to_row("2442"))
results.append(run_test_case_to_row("2002"))
results.append(run_test_case_to_row("1001"))

# 4. Keypad and L/walk patterns
results.append(run_test_case_to_row("2580"))
results.append(run_test_case_to_row("1478"))
results.append(run_test_case_to_row("3690"))
results.append(run_test_case_to_row("963852"))
results.append(run_test_case_to_row("147852"))
results.append(run_test_case_to_row("159357"))
results.append(run_test_case_to_row("951753"))
results.append(run_test_case_to_row("123789"))
results.append(run_test_case_to_row("741852"))
results.append(run_test_case_to_row("7852"))

# 5. Year-based and date-based PINs
results.append(run_test_case_to_row("2000"))
results.append(run_test_case_to_row("1999"))
results.append(run_test_case_to_row("2024"))
results.append(run_test_case_to_row("2025"))
results.append(run_test_case_to_row("1984"))
results.append(run_test_case_to_row("2015"))
results.append(run_test_case_to_row("0307", dob="03-07-1994"))
results.append(run_test_case_to_row("0703", dob="03-07-1994"))
results.append(run_test_case_to_row("9403", dob="03-07-1994"))
results.append(run_test_case_to_row("1994", dob="03-07-1994"))
results.append(run_test_case_to_row("071994", dob="03-07-1994"))
results.append(run_test_case_to_row("030794", dob="03-07-1994"))
results.append(run_test_case_to_row("0506", spouse_dob="05-06-1997"))
results.append(run_test_case_to_row("0605", spouse_dob="05-06-1997"))
results.append(run_test_case_to_row("9705", spouse_dob="05-06-1997"))
results.append(run_test_case_to_row("1997", spouse_dob="05-06-1997"))
results.append(run_test_case_to_row("051997", spouse_dob="05-06-1997"))
results.append(run_test_case_to_row("050697", spouse_dob="05-06-1997"))
results.append(run_test_case_to_row("0101", anniversary="01-01-2015"))
results.append(run_test_case_to_row("2015", anniversary="01-01-2015"))
results.append(run_test_case_to_row("010120", anniversary="01-01-2020"))
results.append(run_test_case_to_row("200101", anniversary="01-01-2001"))

# 6. Phone number overlap
results.append(run_test_case_to_row("1234", phone="9876123450"))
results.append(run_test_case_to_row("5678", phone="4567898765"))
results.append(run_test_case_to_row("0000", phone="9000001234"))
results.append(run_test_case_to_row("8888", phone="1234888899"))
results.append(run_test_case_to_row("6789", phone="001123456789"))

# 7. Strong, random, and edge PINs
results.append(run_test_case_to_row("3851"))
results.append(run_test_case_to_row("4397"))
results.append(run_test_case_to_row("928351"))
results.append(run_test_case_to_row("841729"))
results.append(run_test_case_to_row("673241"))
results.append(run_test_case_to_row("319472", dob="17-03-1994", spouse_dob="05-06-1997", anniversary="01-01-2015", phone="9876543210"))
results.append(run_test_case_to_row("4051", dob="17-03-1994", phone="1112223334"))
results.append(run_test_case_to_row("7429", spouse_dob="05-06-1997", phone="8765432198"))
results.append(run_test_case_to_row("628319", anniversary="01-01-2015", phone="1234567890"))

# 8. Edge cases: minimal, maximal, odd formats, zeros
results.append(run_test_case_to_row("0001"))
results.append(run_test_case_to_row("1000"))
results.append(run_test_case_to_row("999999"))
results.append(run_test_case_to_row("000999"))
results.append(run_test_case_to_row("009900"))
results.append(run_test_case_to_row("099900"))
results.append(run_test_case_to_row("900900"))
results.append(run_test_case_to_row("090909"))
results.append(run_test_case_to_row("009900", dob="09-09-2000"))
results.append(run_test_case_to_row("123456", dob="15-08-1996", spouse_dob="13-12-1998", anniversary="31-12-2020", phone="1234567890"))
results.append(run_test_case_to_row("2853", dob="15-08-1996", phone=""))

# 9. Invalid and boundary values
results.append(run_test_case_to_row("12a4"))   # invalid, should return error
results.append(run_test_case_to_row("123"))    # invalid, too short
results.append(run_test_case_to_row("12345"))  # invalid, not 4/6 digits
results.append(run_test_case_to_row("abcdef")) # invalid, not digits

# Create and display the table
df = pd.DataFrame(results)
df = df[["MPIN", "DOB", "Spouse DOB", "Anniv", "Phone", "Strength", "Reasons"]]

from IPython.display import display
display(df.style.set_properties(
    **{
        'background-color': '#f6f9ff',
        'border': '1px solid #2563eb22',
        'border-radius': '8px',
        'color': '#111927',
        'font-family': 'Segoe UI, Arial, sans-serif'
    }
).set_table_attributes('style="width:100%; border-collapse:collapse;"'))

,MPIN,DOB,Spouse DOB,Anniv,Phone,Strength,Reasons
0,1234,,,,,WEAK,SEQUENTIAL_PATTERN
1,2345,,,,,WEAK,SEQUENTIAL_PATTERN
2,6789,,,,,WEAK,SEQUENTIAL_PATTERN
3,7890,,,,,WEAK,SEQUENTIAL_PATTERN
4,4321,,,,,WEAK,SEQUENTIAL_PATTERN
5,2109,,,,,WEAK,SEQUENTIAL_PATTERN
6,0987,,,,,WEAK,SEQUENTIAL_PATTERN
7,3210,,,,,WEAK,SEQUENTIAL_PATTERN
8,123456,,,,,WEAK,SEQUENTIAL_PATTERN
9,654321,,,,,WEAK,SEQUENTIAL_PATTERN
